This is an example of [Litematica](https://litematica.org/) files reading and using via mcms project

Files can be found here: [https://abfielder.com/browseSchematics](https://abfielder.com/browseSchematics)

In [ ]:
# If you have not installed the required packages yet
%pip install tqdm
%pip install litemapy

In [ ]:
# Import section
from typing import Any, List, Optional

import tqdm
from litemapy import BlockState, Region, Schematic

from mcms.connection import Connection
from mcms.data import Block, Location

In [ ]:
# Setup connection
connection = Connection("localhost", 8000, ("Admin", "123"))

In [ ]:
# Patch to fix issue with rotation (litemapy -> minecraft.py -> Entity -> __init__)

# self._rotation = (rotation[0], rotation[1], rotation[2]) -> instead of this

# Use this:
# if len(rotation) == 3:
#     self._rotation = (rotation[0], rotation[2], rotation[1])
# elif len(rotation) == 2:
#     self._rotation = (rotation[0], rotation[1], 0)
# elif len(rotation) == 1:
#     self._rotation = (rotation[0], 0, 0)
# else:
#     self._rotation = (0, 0, 0)

In [ ]:
def modify_start_location(location: Location, region: Region) -> Location:
    return Location(x=location.x - region.min_x()-1, y=location.y - region.min_y()-1, z=location.z - region.min_z()-1)

In [ ]:
def to_block(block_state: BlockState, location: Location) -> Block:
    namespace, name = block_state.id.split(":")
    state = {}
    nbt = block_state.to_nbt()
    if "Properties" in nbt:
        for k, v in nbt["Properties"].items():
            state[str(k)] = str(v)
    return Block(namespace=namespace, name=name, location=location, state=state)

In [ ]:
def create_blocks(region: Region, start: Location, default_block_name: Optional[str] = None) -> List[Block]:
    start_location = modify_start_location(start, region)
    blocks = []
    for x in region.xrange():
        for y in region.yrange():
            for z in region.zrange():
                block_state = region[x, y, z]

                if block_state.id == "minecraft:air":
                    continue

                location = Location(x=x + start_location.x, y=y + start_location.y, z=z + start_location.z)

                if default_block_name is not None:
                    block = Block(name=default_block_name, location=location)
                else:
                    block = to_block(block_state, location)

                blocks.append(block)
    return blocks

In [ ]:
def fill(block: Block, x_length: int = 1, y_length: int = 1, z_length: int = 1) -> List[Block]:
    blocks = []
    for x in range(0, x_length):
        for y in range(0, y_length):
            for z in range(0, z_length):
                blocks.append(Block(
                    namespace=block.namespace,
                    name=block.name,
                    location=Location(x=block.location.x + x, y=block.location.y + y, z=block.location.z + z),
                    state=block.state,
                    inventory=block.inventory))
    return blocks

In [ ]:
def clear_space_near_to_player(
        connection: Connection,
        player_name: str,
        x_length: int = 1,
        y_length: int = 1,
        z_length: int = 1,
        timeout: int = 100) -> int:

    # Get the start location
    players = connection.get_players(only_online=True)
    players = {p.name: p for p in players[1]}
    start_location = players[player_name].location

    start_block = Block(name="air", location=Location(x=int(start_location.x), y=int(start_location.y), z=int(start_location.z)))
    blocks = fill(start_block, x_length, y_length, z_length)

    result = connection.set_blocks(blocks, timeout=timeout)
    return result[0]

In [ ]:
def split_list(my_list: List[Any], sublist_size: int) -> List[List[Any]]:
    return [my_list[i:i+sublist_size] for i in range(0, len(my_list), sublist_size)]

In [ ]:
def build_region(connection: Connection, region: Region, start: Location, split_size: int = 10000, timeout: int = 300) -> int:
    print("Create blocks...")
    blocks = create_blocks(region, start)
    print("Number of blocks:", len(blocks))
    blocks_set = split_list(blocks, split_size)
    print("Start building...")
    for b in tqdm.tqdm(blocks_set, ncols=100):
        result = connection.set_blocks(b, timeout=timeout)
    return result[0]

In [ ]:
def clear_region(connection: Connection, region: Region, start: Location, split_size: int = 10000, timeout: int = 300) -> int:
    print("Create blocks...")
    blocks = create_blocks(region, start, default_block_name="air")
    blocks_set = split_list(blocks, split_size)
    print("Start cleaning...")
    for b in tqdm.tqdm(blocks_set, ncols=100):
        result = connection.set_blocks(b, timeout=timeout)
    return result[0]

In [ ]:
# Load the schematic and get its first region
schema = Schematic.load(r"Path to your file.litematic")

region = next(iter(schema.regions.values()))
print("Number of blocks:", region.count_blocks())
print("Start x coordinate:", region.min_x())
print("Start y coordinate:", region.min_y())
print("Start z coordinate:", region.min_z())

In [ ]:
# Start coordinates
start = Location(x=-210, y=72, z=-565)
# Build the region
build_region(connection, region, start)

In [ ]:
# Clear region
clear_region(connection, region, start)